In [8]:
#import the important library
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json as js
import ast
import folium
import re
import networkx as nx
from datetime import datetime
from datetime import timedelta
import branca.colormap as cm
from branca.colormap import linear
from folium.plugins import TimeSliderChoropleth
import geopandas as gpd

In [9]:
#use the written gird geojson data
state_df = gpd.read_file('us-state.json')

In [10]:
#read the file of tweets by country
covid_tweets_states_df = pd.read_csv("us_states_weekly_counts.csv")
del covid_tweets_states_df['Unnamed: 0']

In [13]:
joined_df = pd.merge(covid_tweets_states_df,state_df,how = 'outer',left_on = 'States',right_on = 'id')
del joined_df['id']
joined_df

,States,Count,Week_index,Date_Sec,name,geometry
0,CA,691.000000,0,1584590400,California,"POLYGON ((-123.23326 42.00619, -122.37885 42.0..."
1,CA,531.571429,1,1585195200,California,"POLYGON ((-123.23326 42.00619, -122.37885 42.0..."
2,CA,383.428571,2,1585800000,California,"POLYGON ((-123.23326 42.00619, -122.37885 42.0..."
3,CA,229.285714,3,1586404800,California,"POLYGON ((-123.23326 42.00619, -122.37885 42.0..."
4,CA,824.857143,4,1587009600,California,"POLYGON ((-123.23326 42.00619, -122.37885 42.0..."
...,...,...,...,...,...,...
2395,WY,10.428571,45,1611810000,Wyoming,"POLYGON ((-109.08084 45.00207, -105.91517 45.0..."
2396,WY,10.285714,46,1612414800,Wyoming,"POLYGON ((-109.08084 45.00207, -105.91517 45.0..."
2397,WY,4.285714,47,1613019600,Wyoming,"POLYGON ((-109.08084 45.00207, -105.91517 45.0..."
2398,WY,6.000000,48,1613624400,Wyoming,"POLYGON ((-109.08084 45.00207, -105.91517 45.0..."


In [15]:
#function to get the time in second for the area
def get_date_index(df):
    if df['Week_index'] == 0:
        return int(datetime.strptime('2020-03-19', '%Y-%m-%d').timestamp())
    else:
        timedel = timedelta(weeks = df['Week_index'] )
        time_week = datetime.strptime('2020-03-19', '%Y-%m-%d') + timedel
        return int(time_week.timestamp())
    
joined_df['Date_Sec'] = joined_df.apply(get_date_index,axis = 1)

In [16]:
#create a dictionary to store the average of each grid
state_avg_dict = {}
state_list = joined_df['States'].unique().tolist()
state_idx = range(len(state_list))

for i in state_idx:
    id = state_list[i]
    temp_df = joined_df[joined_df['States'] == id]
    temp_avg = temp_df['Count'].mean()
    state_avg_dict[id] = temp_avg
    
joined_df['State_Avg'] = joined_df['States'].apply(lambda x: state_avg_dict[x])                                       

joined_df

,States,Count,Week_index,Date_Sec,name,geometry,State_Avg
0,CA,691.000000,0,1584590400,California,"POLYGON ((-123.23326 42.00619, -122.37885 42.0...",1513.248571
1,CA,531.571429,1,1585195200,California,"POLYGON ((-123.23326 42.00619, -122.37885 42.0...",1513.248571
2,CA,383.428571,2,1585800000,California,"POLYGON ((-123.23326 42.00619, -122.37885 42.0...",1513.248571
3,CA,229.285714,3,1586404800,California,"POLYGON ((-123.23326 42.00619, -122.37885 42.0...",1513.248571
4,CA,824.857143,4,1587009600,California,"POLYGON ((-123.23326 42.00619, -122.37885 42.0...",1513.248571
...,...,...,...,...,...,...,...
2395,WY,10.428571,45,1611810000,Wyoming,"POLYGON ((-109.08084 45.00207, -105.91517 45.0...",9.554286
2396,WY,10.285714,46,1612414800,Wyoming,"POLYGON ((-109.08084 45.00207, -105.91517 45.0...",9.554286
2397,WY,4.285714,47,1613019600,Wyoming,"POLYGON ((-109.08084 45.00207, -105.91517 45.0...",9.554286
2398,WY,6.000000,48,1613624400,Wyoming,"POLYGON ((-109.08084 45.00207, -105.91517 45.0...",9.554286


In [20]:
#create a function to get the percentage change in the value
def get_the_change_percentage(df):
    
    current_state_avg = df['State_Avg']
    
    #if the avg is zero which means there is nothing in the places
    if current_state_avg == 0.0:
        return 0.0
    else:
        current_state_count =df['Count']
        delta = (current_state_count - current_state_avg)/current_state_avg*100
        return delta
    
joined_df['Normalized_change']=joined_df.apply(get_the_change_percentage,axis = 1) 

In [22]:
#create geo info stirng
pls_df = joined_df[['geometry']]
pls_gdf = gpd.GeoDataFrame(pls_df)
pls_gdf = pls_gdf.drop_duplicates().reset_index() 

In [29]:
#function to get the opacity of the area
def get_opacity(df):
    if df['Count'] == 0:
        return 0.00
    else:
        return 0.7
joined_df['Opacity'] = joined_df.apply(get_opacity,axis = 1)

#get the color code
max_colour = max(joined_df['Count'])
min_colour = min(joined_df['Count'])
cmap = cm.linear.YlOrRd_07.scale(min_colour, max_colour)
joined_df['Colour'] = joined_df['Count'].map(cmap)

#create style dictionary
idf_list = joined_df['States'].unique().tolist()
idf_idx = range(len(idf_list))

style_dict = {}
for i in idf_idx:
    id = idf_list[i]
    result = joined_df[joined_df['States'] == id]
    inner_dict = {}
    for c, r in result.iterrows():
        inner_dict[r['Date_Sec']] = {'color': r['Colour'], 'opacity': r['Opacity']}
    style_dict[str(i)] = inner_dict


In [30]:
#create and add the info into the bar
slider_map = folium.Map(location=[36.648693, -95.8974445],max_bounds=True,tiles = 'cartodbpositron',zoom_control=False, zoom_start=4)

widget = TimeSliderChoropleth( 
    data=pls_gdf.to_json(),
    styledict=style_dict,

).add_to(slider_map)
cmap.caption = 'Number of tweets in the area'
widget = cmap.add_to(slider_map)

In [31]:
slider_map

In [41]:
#function to get the opacity of the area
def get_opacity(df):
    if df['Normalized_change'] == 0:
        return 0.00
    else:
        return 0.7
joined_df['Opacity'] = joined_df.apply(get_opacity,axis = 1)
joined_df


#get the color code
max_colour = max(joined_df['Normalized_change'])
min_colour = min(joined_df['Normalized_change'])
cmap = cm.linear.RdBu_11.scale(min_colour, max_colour)
joined_df['Colour'] = joined_df['Normalized_change'].map(cmap)

#create style dictionary
idf_list = joined_df['States'].unique().tolist()
idf_idx = range(len(idf_list))

style_dict = {}
for i in idf_idx:
    id = idf_list[i]
    result = joined_df[joined_df['States'] == id]
    inner_dict = {}
    for c, r in result.iterrows():
        inner_dict[r['Date_Sec']] = {'color': r['Colour'], 'opacity': r['Opacity']}
    style_dict[str(i)] = inner_dict


In [42]:
#create and add the info into the bar
slider_map = folium.Map(location=[36.648693, -95.8974445],max_bounds=True,tiles = 'cartodbpositron',zoom_control=False, zoom_start=4)

widget = TimeSliderChoropleth( 
    data=pls_gdf.to_json(),
    styledict=style_dict,

).add_to(slider_map)
cmap.caption = 'Pertcent of change in the number of tweets in the area'
widget = cmap.add_to(slider_map)

In [43]:
slider_map